## Bibliotecas e dependencias

In [ ]:
!pip install kagglehub[pandas-datasets] gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
import json
import re
import random
from time import sleep
import gradio as gr
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [ ]:
from google.colab import userdata
key = userdata.get('openAIkey')

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=key)

## Pre-processamento de dados

In [ ]:
hamlet_df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "umerhaddii/shakespeare-plays-dialogues",
  "hamlet.csv",
)

macbeth_df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "umerhaddii/shakespeare-plays-dialogues",
  "macbeth.csv",
)

romeo_juliet_df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "umerhaddii/shakespeare-plays-dialogues",
  "romeo_juliet.csv",
)


<ipython-input-8-99f355316fa4>:1: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  hamlet_df = kagglehub.load_dataset(


100%|██████████| 289k/289k [00:00<00:00, 1.17MB/s]
<ipython-input-8-99f355316fa4>:7: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  macbeth_df = kagglehub.load_dataset(
<ipython-input-8-99f355316fa4>:13: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  romeo_juliet_df = kagglehub.load_dataset(


In [ ]:
hamlet_df.head()

,act,scene,character,dialogue,line_number
0,Act I,Scene I,[stage direction],FRANCISCO at his post. Enter to him BERNARDO,NaN
1,Act I,Scene I,Bernardo,Who's there?,1.0
2,Act I,Scene I,Francisco,"Nay, answer me: stand, and unfold yourself.",2.0
3,Act I,Scene I,Bernardo,Long live the king!,3.0
4,Act I,Scene I,Francisco,Bernardo?,4.0


In [ ]:
macbeth_df.head()

,act,scene,character,dialogue,line_number
0,Act I,Scene I,[stage direction],Thunder and lightning. Enter three Witches,NaN
1,Act I,Scene I,First Witch,When shall we three meet again,1.0
2,Act I,Scene I,First Witch,"In thunder, lightning, or in rain?",2.0
3,Act I,Scene I,Second Witch,"When the hurlyburly's done,",3.0
4,Act I,Scene I,Second Witch,When the battle's lost and won.,4.0


In [ ]:
romeo_juliet_df.head()

,act,scene,character,dialogue,line_number
0,Act I,Prologue,Chorus,"Two households, both alike in dignity,",1.0
1,Act I,Prologue,Chorus,"In fair Verona, where we lay our scene,",2.0
2,Act I,Prologue,Chorus,"From ancient grudge break to new mutiny,",3.0
3,Act I,Prologue,Chorus,Where civil blood makes civil hands unclean.,4.0
4,Act I,Prologue,Chorus,From forth the fatal loins of these two foes,5.0


In [ ]:
# Função para agrupar por cena e formatar corretamente
def format_scene(df_scene):
    """Formata uma cena inteira mantendo personagens e direções de palco"""
    formatted_scene = []

    for _, row in df_scene.iterrows():
        character = row['character']
        dialogue = row['dialogue']

        if character == "[stage direction]":
            # Formatar direções de palco em itálico
            formatted_scene.append(f"[{dialogue}]")
        else:
            # Formatar fala de personagem
            formatted_scene.append(f"{character}: {dialogue}")

    return "\n".join(formatted_scene)

In [ ]:
# Função para extrair monólogos (falas longas de um personagem)
def extract_monologues(df, min_lines=5):
    """Encontra monólogos (falas consecutivas de um mesmo personagem)"""
    monologues = []
    current_character = None
    current_monologue = []

    for _, row in df.iterrows():
        character = row['character']
        dialogue = row['dialogue']

        # Pular direções de palco
        if character == "[stage direction]":
            continue

        # Se mudar de personagem, verificar se temos um monólogo
        if character != current_character:
            if current_character and len(current_monologue) >= min_lines:
                monologues.append({
                    "character": current_character,
                    "text": "\n".join(current_monologue)
                })
            current_character = character
            current_monologue = [dialogue]
        else:
            current_monologue.append(dialogue)

    # Verificar o último monólogo
    if current_character and len(current_monologue) >= min_lines:
        monologues.append({
            "character": current_character,
            "text": "\n".join(current_monologue)
        })

    return monologues

In [ ]:
# Criar exemplos de fine-tuning para cenas completas
def create_scene_examples(df):
    examples = []

    # Agrupar por ato e cena
    grouped = df.groupby(['act', 'scene'])

    for (act, scene), group in grouped:
        formatted_scene = format_scene(group)

        # Criar um tema baseado no conteúdo da cena
        if "murder" in formatted_scene.lower() or "kill" in formatted_scene.lower():
            tema = "assassinato e vingança"
        elif "love" in formatted_scene.lower():
            tema = "amor proibido"
        elif "ghost" in formatted_scene.lower():
            tema = "encontro com um fantasma"
        elif "witch" in formatted_scene.lower():
            tema = "bruxaria e premonição"
        else:
            tema = "conflito entre personagens nobres"

        # Criar exemplo no formato JSONL
        example = {
            "messages": [
                {"role": "system", "content": "Você é William Shakespeare, o dramaturgo. Escreva no estilo autêntico de suas peças, usando linguagem arcaica, riqueza vocabular, estruturas dramáticas precisas (pentâmetro iâmbico, verso livre) e indicações de palco, metáforas, trocadilhos."},
                {"role": "user", "content": f"Escreva uma cena shakespeariana sobre {tema}."},
                {"role": "assistant", "content": formatted_scene}
            ]
        }
        examples.append(example)

    return examples

In [ ]:
# Criar exemplos de fine-tuning para monólogos
def create_monologue_examples(df):
    examples = []
    monologues = extract_monologues(df)

    for mono in monologues:
        character = mono["character"]
        text = mono["text"]

        # Criar tema com base no conteúdo do monólogo
        if "be or not to be" in text.lower():
            tema = "contemplação da morte"
        elif "wherefore art thou" in text.lower():
            tema = "amor impossível"
        elif "dagger" in text.lower():
            tema = "culpa e alucinação"
        else:
            # Temas genéricos para outros monólogos
            temas = ["reflexão filosófica", "dilema moral", "lamento pessoal",
                    "ambição desmedida", "desespero humano", "confronto com o destino"]
            tema = random.choice(temas)

        example = {
            "messages": [
                {"role": "system", "content": "Você é William Shakespeare. Escreva monólogos dramáticos no seu estilo característico, usando pentâmetro iâmbico e linguagem arcaica rica em metáforas."},
                {"role": "user", "content": f"Escreva um monólogo shakespeariano para um personagem contemplando {tema}."},
                {"role": "assistant", "content": f"{character}:\n{text}"}
            ]
        }
        examples.append(example)

    return examples

In [ ]:
# Combinar exemplos de todas as peças
all_examples = []

# Cenas
all_examples.extend(create_scene_examples(hamlet_df))
all_examples.extend(create_scene_examples(macbeth_df))
all_examples.extend(create_scene_examples(romeo_juliet_df))

# Monólogos
all_examples.extend(create_monologue_examples(hamlet_df))
all_examples.extend(create_monologue_examples(macbeth_df))
all_examples.extend(create_monologue_examples(romeo_juliet_df))

# Limitar exemplos se necessário
if len(all_examples) > 100:
    all_examples = random.sample(all_examples, 100)

# Salvar no formato JSONL
with open("shakespeare_plays_finetuning.jsonl", "w") as f:
    for example in all_examples:
        f.write(json.dumps(example) + "\n")

print(f"Criados {len(all_examples)} exemplos para fine-tuning")

Criados 100 exemplos para fine-tuning


In [ ]:
# Exibir 3 exemplos aleatórios para verificação

sample_examples = random.sample(all_examples, 3)
for i, example in enumerate(sample_examples):
    print(f"\n--- Exemplo {i+1} ---")
    print(f"Sistema: {example['messages'][0]['content']}")
    print(f"Usuário: {example['messages'][1]['content']}")
    print(f"Assistente (primeiras 100 chars): {example['messages'][2]['content'][:100]}...")


--- Exemplo 1 ---
Sistema: Você é William Shakespeare, o dramaturgo. Escreva no estilo autêntico de suas peças, usando linguagem arcaica, riqueza vocabular, estruturas dramáticas precisas (pentâmetro iâmbico, verso livre) e indicações de palco, metáforas, trocadilhos.
Usuário: Escreva uma cena shakespeariana sobre amor proibido.
Assistente (primeiras 100 chars): [Enter ROMEO, MERCUTIO, BENVOLIO, with five or six Maskers, Torch-bearers, and others]
Romeo: What, ...

--- Exemplo 2 ---
Sistema: Você é William Shakespeare. Escreva monólogos dramáticos no seu estilo característico, usando pentâmetro iâmbico e linguagem arcaica rica em metáforas.
Usuário: Escreva um monólogo shakespeariano para um personagem contemplando culpa e alucinação.
Assistente (primeiras 100 chars): Lady Macbeth:
O proper stuff!
This is the very painting of your fear:
This is the air-drawn dagger w...

--- Exemplo 3 ---
Sistema: Você é William Shakespeare. Escreva monólogos dramáticos no seu estilo característico, u

## Fine-tuning

In [ ]:
def monitor_job(job_id):
    """Monitor fine-tuning job progress"""
    while True:
        job = client.fine_tuning.jobs.retrieve(job_id)
        print(f"Status: {job.status}")

        if job.status in ["succeeded", "failed"]:
            return job

        # List latest events
        events = client.fine_tuning.jobs.list_events(
            fine_tuning_job_id=job_id,
            limit=5
        )
        for event in events.data:
            print(f"Event: {event.message}")

        sleep(20)  # Check every 20 seconds


In [ ]:
# Upload do arquivo JSONL
with open("/content/shakespeare_plays_finetuning.jsonl", "rb") as file:
    response = client.files.create(
        file=file,
        purpose="fine-tune"
    )
    file_id = response.id
    print(f"Arquivo carregado com ID: {file_id}")

# Criar o job de fine-tuning
create_job = client.fine_tuning.jobs.create(
    training_file=file_id,
    model="gpt-4o-mini-2024-07-18",  # Custo-benefício melhor
    method={
        "type": "supervised",
        "supervised": {
            "hyperparameters": {
                "n_epochs": 2,
                "learning_rate_multiplier": 0.2,
                "batch_size": 16
            },
        }
    }
)

job_id = create_job.id
print(f"Fine-tuning iniciado! Job ID: {job_id}")

Arquivo carregado com ID: file-ASGAgvbDygBte4QF6mTnAd
Fine-tuning iniciado! Job ID: ftjob-c01wo6xYyMqiSyUIkkVfyee6


In [ ]:
# Monitor the job until completion
curr_job = monitor_job(job_id)
if curr_job.status == "succeeded":
    fine_tuned_model = curr_job.fine_tuned_model
    print(f"Fine-tuned model ID: {fine_tuned_model}")
else:
    print("Fine-tuning falhou.")


Status: validating_files
Event: Validating training file: file-ASGAgvbDygBte4QF6mTnAd
Event: Created fine-tuning job: ftjob-c01wo6xYyMqiSyUIkkVfyee6
Status: validating_files
Event: Validating training file: file-ASGAgvbDygBte4QF6mTnAd
Event: Created fine-tuning job: ftjob-c01wo6xYyMqiSyUIkkVfyee6
Status: validating_files
Event: Validating training file: file-ASGAgvbDygBte4QF6mTnAd
Event: Created fine-tuning job: ftjob-c01wo6xYyMqiSyUIkkVfyee6
Status: validating_files
Event: Validating training file: file-ASGAgvbDygBte4QF6mTnAd
Event: Created fine-tuning job: ftjob-c01wo6xYyMqiSyUIkkVfyee6
Status: validating_files
Event: Validating training file: file-ASGAgvbDygBte4QF6mTnAd
Event: Created fine-tuning job: ftjob-c01wo6xYyMqiSyUIkkVfyee6
Status: validating_files
Event: Validating training file: file-ASGAgvbDygBte4QF6mTnAd
Event: Created fine-tuning job: ftjob-c01wo6xYyMqiSyUIkkVfyee6
Status: validating_files
Event: Validating training file: file-ASGAgvbDygBte4QF6mTnAd
Event: Created fine-

## Criacao, Execucao da Aplicação e Testes dos Modelos

Models:

 - fine-tunados com 20 exemplos do dataset umerhaddii/shakespeare-plays-dialogues
    1. ft:gpt-4o-mini-2024-07-18:hendrik::BJ8DAYRn  
      - overfitting
      - epochs: 3
    2. ft:gpt-4o-mini-2024-07-18:hendrik::BJ95WQdI
      - epochs: 1, learning_rate: 0.1
    3. ft:gpt-4o-mini-2024-07-18:hendrik::BJa2oeL5
      - epochs: 1, learning_rate: 0.1, batch_size = 8
  
  
  - fine-tunados com 50 exemplos do dataset umerhaddii/shakespeare-plays-dialogues
    4. ft:gpt-4o-mini-2024-07-18:hendrik::BJkjPlqi
      - epochs: 1, learning_rate: 0.15, batch_size: 8
  - fine-tunados com 100 exemplos do dataset umerhaddii/shakespeare-plays-dialogues
    5. ft:gpt-4o-mini-2024-07-18:hendrik::BJkuBW58
      - epochs: 2, learning_rate: 0.2, batch_size: 16

In [ ]:
model1 = "ft:gpt-4o-mini-2024-07-18:hendrik::BJ8DAYRn"
model2 = "ft:gpt-4o-mini-2024-07-18:hendrik::BJ95WQdI"
model3 = "ft:gpt-4o-mini-2024-07-18:hendrik::BJa2oeL5"
model4 = "ft:gpt-4o-mini-2024-07-18:hendrik::BJkjPlqi"
model5 = "ft:gpt-4o-mini-2024-07-18:hendrik::BJkuBW58"

In [ ]:
def gerar_texto_shakespeare(tema, tipo_texto="cena", personagens=2, temperatura=0.8, model="gpt-4"):
    """Usa o modelo fine-tuned para gerar texto shakespeariano"""
    max_tks = 1500
    # Adaptar prompt com base no tipo de texto
    if tipo_texto == "cena":
        content = f"Crie uma cena original e nova (nova=que nao consta nas obras de shakespeare) no estilo autêntico de Shakespeare sobre '{tema}' com {personagens} personagens novos e originais. Incorpore linguagem e vocaulário: uso de arcaísmos, riqueza vocabular; estrutura métrica: pentâmetro iâmbico, verso livre; e figuras de linguagem e retórica: metáforas, trocadilhos."
        max_tks = 2000
    elif tipo_texto == "monologo":
        content = f"Crie um monólogo original e novo (novo=que nao consta nas obras de shakespeare) poderoso no estilo de Shakespeare sobre '{tema}'. Incorpore linguagem e vocaulário: uso de arcaísmos, riqueza vocabular; estrutura métrica: pentâmetro iâmbico, verso livre; e figuras de linguagem e retórica: metáforas, trocadilhos."
        max_tks = 1200
    else:
        content = f"Crie uma cena original e nova (nova=que nao consta nas obras de shakespeare) no estilo autêntico de Shakespeare sobre '{tema}' com {personagens} personagens novos e originais. Incorpore linguagem e vocaulário: uso de arcaísmos, riqueza vocabular; estrutura métrica: pentâmetro iâmbico, verso livre; e figuras de linguagem e retórica: metáforas, trocadilhos."

    # Chamada à API com o modelo fine-tuned
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Você é William Shakespeare, o dramaturgo."},
            {"role": "user", "content": content}
        ],
        temperature = temperatura,
        max_tokens=max_tks
    )

    return response.choices[0].message.content

In [ ]:
def analisar_estilo_shakespeare(texto_gerado):
    """Analisa o quão "shakespeariano" é o texto gerado"""

    prompt_analise = f"""Analise o seguinte texto e avalie quanto ele se assemelha ao estilo autêntico de Shakespeare.

    TEXTO:
    {texto_gerado[:2500]}... (truncado)

    Forneça uma análise detalhada considerando (escala de 1-10 para cade criterio):
    1. Linguagem e vocabulário (uso de arcaísmos, riqueza vocabular)
    2. Estrutura métrica (pentâmetro iâmbico, verso livre)
    3. Figuras de linguagem e retórica (metáforas, trocadilhos)

    4. Realize uma média dos criterios de autenticidade avaliados.

    Realize uma análise sucinta, pórem precisa.
    Destaque apenas os principais pontos.
    Não leve em consideração a análise de temas e elementos sobre os quais Shakespeare nunca escreveu antes!
    Se necessário, inclua exemplos específicos do texto que justifiquem sua avaliação.
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um especialista em literatura shakespeariana."},
            {"role": "user", "content": prompt_analise}
        ],
        temperature=0.3,
        max_tokens=1000
    )

    return response.choices[0].message.content

In [ ]:
def gerar_imagem_shakespeare(texto, cena_titulo):
    """Gera uma imagem representando uma cena ou personagem da peça"""

    # Extrair uma descrição concisa para a imagem
    prompt_descricao = f"""
    Com base neste texto de peça shakespeariana:
    {texto[:1000]}...

    Crie uma descrição visual concisa (máximo 50 palavras) para uma imagem dramática
    representando a cena "{cena_titulo}". A descrição deve capturar a essência dramática da cena.
    """

    descricao_response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um artista renascentista."},
            {"role": "user", "content": prompt_descricao}
        ],
        temperature=0.8,
        max_tokens=100
    )

    descricao_imagem = descricao_response.choices[0].message.content

    # Gerar a imagem com DALL-E
    imagem_response = client.images.generate(
        prompt=f"Uma ilustração dramática que represente bem o contexto: {descricao_imagem}",
        n=1,
        size="1024x1024"
    )

    return imagem_response.data[0].url, descricao_imagem

In [ ]:
def criar_aplicacao_shakespeare():
    """Cria a interface Gradio para o DeepFake Shakespeare"""

    # Função que conecta todos os componentes
    def processar_pedido(tema, tipo_texto, num_personagens):
        # Gerar texto Shakespeare
        texto_gerado = gerar_texto_shakespeare(tema, tipo_texto, num_personagens, model=model5) # Altere o modelo para o Gradio aqui
        #print(texto_gerado)

        # Analisar autenticidade
        analise = analisar_estilo_shakespeare(texto_gerado)

        # Gerar imagem
        titulo_cena = f"{tema} - {tipo_texto} shakespeariano"
        url_imagem, descricao_imagem = gerar_imagem_shakespeare(texto_gerado, titulo_cena)

        return texto_gerado, analise, url_imagem, descricao_imagem

    # Interface Gradio
    with gr.Blocks(title="DeepFake Shakespeare: O Teatro que Nunca Existiu") as app:
        gr.Markdown("# 🎭 DeepFake Shakespeare: O Teatro que Nunca Existiu")
        gr.Markdown("Recrie peças shakespearianas que Shakespeare nunca escreveu, mas que poderiam ter existido!")

        with gr.Row():
            with gr.Column():
                tema_input = gr.Textbox(label="Tema da Peça", placeholder="Ex: A Revolução Francesa, Napoleão, etc.")
                tipo_text = gr.Radio(
                    ["cena", "monologo"],
                    label="Tipo de Texto",
                    value="cena"
                )
                num_personagens = gr.Slider(
                    minimum=1,
                    maximum=5,
                    value=2,
                    step=1,
                    label="Número de Personagens"
                )
                submit_btn = gr.Button("Gerar Obra Shakespeariana")

            with gr.Column():
                img_output = gr.Image(label="Visualização da Cena")
                img_desc = gr.Textbox(label="Descrição da Imagem")

        with gr.Row():
            texto_output = gr.Textbox(label="Texto Gerado", lines=15)
            analise_output = gr.Textbox(label="Análise de Autenticidade", lines=15)

        submit_btn.click(
            processar_pedido,
            inputs=[tema_input, tipo_text, num_personagens],
            outputs=[texto_output, analise_output, img_output, img_desc]
        )

        gr.Markdown("## 📝 Sobre o Projeto")
        gr.Markdown("Este projeto explora o limite da originalidade da IA, recriando obras que Shakespeare nunca escreveu, mas que poderiam ter existido. Usa GPT-4o-mini com fine-tuning para gerar textos no estilo shakespeariano e DALL-E para visualizar as cenas.")

    return app

In [ ]:
# Iniciar a aplicação
demo = criar_aplicacao_shakespeare()
# share=True cria um lanca uma demo e cria um link publico, inline=True pra visualizar no colab/notebook
demo.launch(share=True, inline=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4ed31b56c4b8c810d0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Testando e avaliando diferentes temperaturas e modelos para a geração de texto

In [ ]:
resultados_modelgpt4 = {}
for temp in [0.8]:
    resultados_modelgpt4[f"temp_{temp}"] = gerar_texto_shakespeare(
        "A Revolução Industrial",
        temperatura=temp,
    )

In [ ]:
# for k,v in resultados_modelgpt4.items():
#   print()
#   print(k)
#   print(v[:200])

In [ ]:
analises_modelgpt4 = {}
for temp in [0.8]:
    analises_modelgpt4[f"temp_{temp}"] = analisar_estilo_shakespeare(
        resultados_modelgpt4[f"temp_{temp}"]
    )

In [ ]:
for k,v in analises_modelgpt4.items():
  print()
  print(k)
  print(v)


temp_0.8
1. Linguagem e Vocabulário: A linguagem e o vocabulário do texto são ricos e poéticos, com um uso de arcaísmos que evocam o estilo de Shakespeare. No entanto, o texto também inclui referências a elementos modernos, como a Revolução Industrial, que não estariam presentes na obra de Shakespeare. Ainda assim, o uso de palavras como "fumegantes", "fuligem" e "fornalhas" são reminiscentes do vocabulário shakespeariano. Avaliação: 7/10.

2. Estrutura Métrica: O texto não segue estritamente o pentâmetro iâmbico, que é uma característica marcante da obra de Shakespeare. No entanto, há uma tentativa de manter um ritmo poético e um padrão de rima ABAB que é comum na poesia shakespeariana. Avaliação: 6/10.

3. Figuras de Linguagem e Retórica: O texto faz uso de metáforas, como "O que é um homem, senão forja de si mesmo", que é uma figura de linguagem comum em Shakespeare. No entanto, não há muitos trocadilhos, que são uma característica proeminente do estilo de Shakespeare. Avaliação: 7

In [ ]:
resultados_model2 = {}
for temp in [0.8]:
    resultados_model2[f"temp_{temp}"] = gerar_texto_shakespeare(
        "A Revolução Industrial",
        temperatura=temp,
        model=model2
    )

In [ ]:
# for k,v in resultados_model2.items():
#   print()
#   print(k)
#   print(v[:200])

In [ ]:
analises_model2 = {}
for temp in [0.8]:
    analises_model2[f"temp_{temp}"] = analisar_estilo_shakespeare(
        resultados_model2[f"temp_{temp}"]
    )

In [ ]:
for k,v in analises_model2.items():
  print()
  print(k)
  print(v)


temp_0.8
1. Linguagem e vocabulário: 7/10. O texto faz uso de um vocabulário rico e arcaico, com palavras e expressões que remetem ao estilo shakespeariano. No entanto, há uma certa modernidade em alguns termos e conceitos que não se encaixam perfeitamente na época de Shakespeare.

2. Estrutura métrica: 6/10. O texto não segue estritamente o pentâmetro iâmbico, uma característica marcante das obras de Shakespeare. No entanto, há uma tentativa de manter um ritmo poético consistente.

3. Figuras de linguagem e retórica: 8/10. O texto faz uso de várias metáforas e imagens poéticas, como "a alma é um fio que se entrelaça" e "o homem se torna máquina". Isso é bastante consistente com o estilo de Shakespeare, que frequentemente usava figuras de linguagem para expressar ideias complexas e emoções.

Média dos critérios de autenticidade avaliados: 7/10. O texto tem muitos elementos que são reminiscentes do estilo de Shakespeare, mas não é uma imitação perfeita. A linguagem e as figuras de ling

In [ ]:
resultados_model3 = {}
for temp in [0.8]:
    resultados_model3[f"temp_{temp}"] = gerar_texto_shakespeare(
        "A Revolução Industrial",
        temperatura=temp,
        model=model3
    )

In [ ]:
# for k,v in resultados_model3.items():
#   print()
#   print(k)
#   print(v[:200])

In [ ]:
analises_model3 = {}
for temp in [0.8]:
    analises_model3[f"temp_{temp}"] = analisar_estilo_shakespeare(
        resultados_model3[f"temp_{temp}"]
    )

In [ ]:
for k,v in analises_model3.items():
  print()
  print(k)
  print(v)


temp_0.8
1. Linguagem e vocabulário: 8/10. O texto faz uso de um vocabulário rico e arcaico, com palavras como "rutilante", "devora" e "roem". No entanto, não é tão arcaico quanto o de Shakespeare, que frequentemente usava palavras e frases que são incomuns ou obsoletas hoje. 

2. Estrutura métrica: 7/10. O texto não segue estritamente o pentâmetro iâmbico, que é a estrutura métrica mais comum nas obras de Shakespeare. No entanto, há uma tentativa de manter um ritmo e uma cadência que são reminiscentes do estilo de Shakespeare.

3. Figuras de linguagem e retórica: 9/10. O texto está repleto de metáforas e imagens poéticas, como "máquinas que roem o coração do homem" e "o choro dos filhos ecoa nas fábricas frias". Isso é muito semelhante ao estilo de Shakespeare, que frequentemente usava metáforas e outros dispositivos retóricos para adicionar profundidade e complexidade a seus textos.

Média dos critérios de autenticidade: 8/10. Embora o texto não siga estritamente a estrutura métrica

In [ ]:
resultados_model4 = {}
for temp in [0.8]:
    resultados_model4[f"temp_{temp}"] = gerar_texto_shakespeare(
        "A Revolução Industrial",
        temperatura=temp,
        model=model4
    )

In [ ]:
# for k,v in resultados_model4.items():
#   print()
#   print(k)
#   print(v[:200])

In [ ]:
analises_model4 = {}
for temp in [0.8]:
    analises_model4[f"temp_{temp}"] = analisar_estilo_shakespeare(
        resultados_model4[f"temp_{temp}"]
    )

In [ ]:
for k,v in analises_model4.items():
  print()
  print(k)
  print(v)


temp_0.8
1. Linguagem e Vocabulário: A linguagem e o vocabulário usados no texto são reminiscentes do estilo de Shakespeare, com o uso de arcaísmos e uma rica variedade de vocabulário. O texto também apresenta uma linguagem poética e dramática, que é uma característica marcante das obras de Shakespeare. No entanto, a linguagem não é tão complexa e intrincada quanto a de Shakespeare, que muitas vezes usava palavras e frases obscuras para adicionar profundidade e subtexto às suas peças. Avaliação: 7/10.

2. Estrutura Métrica: O texto não segue estritamente o pentâmetro iâmbico, que é a métrica mais comum nas obras de Shakespeare. No entanto, há uma tentativa de manter um ritmo poético e uma estrutura de versos, o que é uma característica do estilo de Shakespeare. Avaliação: 6/10.

3. Figuras de Linguagem e Retórica: O texto faz uso de várias figuras de linguagem, incluindo metáforas e simbolismos. Por exemplo, a máquina é usada como uma metáfora para a industrialização e a perda da huma

In [ ]:
resultados_model5 = {}
for temp in [0.8]:
    resultados_model5[f"temp_{temp}"] = gerar_texto_shakespeare(
        "A Revolução Industrial",
        temperatura=temp,
        model=model5
    )

In [ ]:
# for k,v in resultados_model5.items():
#   print()
#   print(k)
#   print(v[:200])

In [ ]:
analises_model5 = {}
for temp in [0.8]:
    analises_model5[f"temp_{temp}"] = analisar_estilo_shakespeare(
        resultados_model5[f"temp_{temp}"]
    )

In [ ]:
for k,v in analises_model5.items():
  print()
  print(k)
  print(v)


temp_0.8
1. Linguagem e vocabulário: 8/10. O texto faz um bom uso de linguagem arcaica e rica em vocabulário, que é característica de Shakespeare. Expressões como "Ah! Que visão horrenda em meu peito se agita!" e "E, no entanto, não sei se a mão do homem É mais terrível que o fogo da máquina" são exemplos disso. No entanto, há algumas palavras e expressões que não são típicas do inglês elisabetano, como "fábrica", "inventor" e "industrialização", que diminuem um pouco a autenticidade.

2. Estrutura métrica: 7/10. Embora o texto não siga estritamente o pentâmetro iâmbico, que é a marca registrada de Shakespeare, há uma tentativa de manter um ritmo poético. Por exemplo, "Eustáquio, conde de novos inventos, Em teu coração pulsa um amor ignóbil!" tem um ritmo que se assemelha ao pentâmetro iâmbico.

3. Figuras de linguagem e retórica: 9/10. O texto está repleto de metáforas e imagens vívidas, que são características do estilo de Shakespeare. Por exemplo, "A revolução é como um rio tempest